In [ ]:
from osgeo import gdal
from datetime import date
import rasterio
import geopandas as gpd
import folium
from sentinelsat.sentinel import SentinelAPI
import numpy as np


In [ ]:
#read any AOI.json

shapefile = gpd.read_file(r'aoi.geojson') #path to geojson file
shp_json = shapefile.to_json()
aoi = None
for i in shapefile['geometry']:
    aoi = i
Map = folium.Map([17.68,83.21],zoom_start=8)
folium.GeoJson(shapefile).add_to(Map)
#Map

In [ ]:
#Connection to Scihub API for Raster Image

username = #myEOusername
password = #EOpassword

sentinal_api = SentinelAPI(username, password, 'https://scihub.copernicus.eu/dhus')
sentinal2 = sentinal_api.query(aoi,date=('20221010', date(2022, 10, 20)),
                     area_relation='Intersects',
                     platformname = 'Sentinel-2',
                     cloudcoverpercentage = (0, 25))
print ("API Passed!")

In [ ]:
raster_collection = list(sentinal2.items())
raster_id = raster_collection[0][0]
product_info = sentinal_api.get_product_odata(raster_id)
if(product_info['Online']==True):
    print("Product is online: Preparing to download")
    #sentinal_api.download(raster_id)
else:
    print("Product is offline: Retry with offline archieve")

In [ ]:
s2_bands = r'E:\MyGIS\Ekatvam\GIS-Algo\LULC\NDVI_test\S2A_MSIL2A_20221019T044811_N0400_R076_T44QQF_20221019T073756\S2A_MSIL2A_20221019T044811_N0400_R076_T44QQF_20221019T073756.SAFE\GRANULE\L2A_T44QQF_A038254_20221019T050236\IMG_DATA\R10m'
red_band = rasterio.open(s2_bands+'\T44QQF_20221019T044811_B04_10m.jp2') # red band
NIR_band = rasterio.open(s2_bands+'\T44QQF_20221019T044811_B08_10m.jp2') # nir band


In [ ]:
with rasterio.open('NDVI_band.tiff','w',driver='Gtiff', width=red_band.width, height=red_band.height, count=3, crs=red_band.crs,transform=red_band.transform, dtype=red_band.dtypes[0]) as rgb:
    rgb.write(red_band.read(1),1) 
    rgb.write(NIR_band.read(1),2) 
    rgb.close()

In [ ]:
output_raster = "clip_img.tiff"
input_raster = "NDVI_band.tiff"

In [ ]:
clip_img = gdal.Warp(output_raster,
               input_raster,
               cutlineDSName = shp_json,
               cutlineLayer = 'extent',
               copyMetadata = True,
               dstNodata = 0)
clip_img = None

In [ ]:
def NDVI(r,nir):
    return ((nir-r)/(nir+r))

In [ ]:
clip_input = gdal.Open("clip_img.tiff")
#nir_b8 = gdal.Open(NIR_band[0])

In [ ]:
red = clip_input.GetRasterBand(1).ReadAsArray().astype(float)
nir = clip_input.GetRasterBand(2).ReadAsArray().astype(float)


In [ ]:
np.seterr(invalid='ignore')
ndvi_calc = NDVI(red, nir)

In [ ]:
x_axis = ndvi_calc.shape[0] 
y_axis = ndvi_calc.shape[1] 


In [ ]:
driver = gdal.GetDriverByName('GTiff')
geo = clip_input.GetGeoTransform()  
proj = clip_input.GetProjection()
ndvi_data = driver.Create("Final_NDVI.tiff",x_axis, y_axis, 1,gdal.GDT_Float32)
ndvi_data.SetGeoTransform( geo )
ndvi_data.SetProjection( proj )
ndvi_data.GetRasterBand(1).WriteArray(ndvi_calc)

In [ ]:
ndvi_final = gdal.Open("Final_NDVI.tiff")
ndvi_final = ndvi_final.GetRasterBand(1).ReadAsArray().astype(float)

In [ ]:
img = folium.raster_layers.ImageOverlay(
        image= ndvi_final,
        bounds=[[50, -100.], [65., -115.]],
        interactive=True,
        cross_origin=False,
        zindex=1,
    )

img.add_to(Map)
#Map